In [25]:
import numpy as np
import pandas as pd
from IPython.display import IFrame

In [26]:
#import streamlit as st
from PIL import Image
# import json
# from Classifier import KNearestNeighbours
from bs4 import BeautifulSoup
import requests,io
import PIL.Image
from urllib.request import urlopen

In [39]:
link = pd.read_csv('resources/data/links.csv')

In [41]:
movie = pd.read_csv('resources/data/movies.csv')

In [43]:
rating = pd.read_csv('resources/data/ratings.csv')

In [44]:
rating.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [42]:
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [45]:
link.head()

,movieId,imdbId,tmdbId,url
0,1,114709,862.0,https://www.imdb.com/title/tt0114709/
1,2,113497,8844.0,https://www.imdb.com/title/tt0113497/
2,3,113228,15602.0,https://www.imdb.com/title/tt0113228/
3,4,114885,31357.0,https://www.imdb.com/title/tt0114885/
4,5,113041,11862.0,https://www.imdb.com/title/tt0113041/


In [46]:
merged = movie.merge(link, how = 'left', on = 'movieId')

In [86]:
merged['year'] = merged['title'].apply(lambda x: x.split(' ')[-1]).str.extract(r'(\d+)')

In [80]:
merged['year'] = merged['title'].str.extract(r'(\d+)')

In [109]:
merged.

,movieId,title,genres,imdbId,tmdbId,url,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,https://www.imdb.com/title/tt0114709/,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,8844.0,https://www.imdb.com/title/tt0113497/,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,113228,15602.0,https://www.imdb.com/title/tt0113228/,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885,31357.0,https://www.imdb.com/title/tt0114885/,1995
4,5,Father of the Bride Part II (1995),Comedy,113041,11862.0,https://www.imdb.com/title/tt0113041/,1995


In [131]:
# df_merged = merged.drop(merged.index[[num for num in merged['year'] if len(str(num)) < 4]], axis = 0) #unique() #.groupby('year').count()

In [129]:
odd = [num for num in merged['year'] if len(str(num)) < 4]

year = list(set([record for record in merged['year'] if record not in odd]))

In [132]:
# merged[merged['year'] == year]

In [97]:
merged[merged['year'] == '101']

,movieId,title,genres,imdbId,tmdbId,url,year
36122,149502,101次求婚,(no genres listed),1674766,187006.0,https://www.imdb.com/title/tt01674766/,101


In [9]:
# data['url'] = data['imdbId'].apply(lambda x: "https://www.imdb.com/title/tt0" + str(x) + '/')

In [10]:
data.shape

(62423, 4)

In [7]:
#pip install streamlit

In [ ]:
https://www.imdb.com/video/vi2052129305?playlistId=tt0114709&ref_=tt_ov_vi
    
https://www.imdb.com/video/vi2171864857/?playlistId=tt0114709?ref_=ext_shr_lnk

In [21]:
# IFrame(width="840", height="472", src="https://youtu.be/8MNJl0L2xe8")

In [4]:
#IFrame(width="840", height="472", src="https://www.youtube.com/embed/NEaUSP4YerM")

In [37]:
def movie_poster_fetcher(imdb_link):
    ## Display Movie Poster
    url_data = requests.get(imdb_link).text
    s_data = BeautifulSoup(url_data, 'html.parser')
    imdb_dp = s_data.find("meta", property="og:image")
    movie_poster_link = imdb_dp.attrs['content']
    u = urlopen(movie_poster_link)
    raw_data = u.read()
    image = PIL.Image.open(io.BytesIO(raw_data))
    image = image.resize((158, 301), )
    image.show()
#     st.image(image, use_column_width=False)

In [133]:
# movie_poster_fetcher(data['url'][0])

In [3]:
#link = "https://www.imdb.com/title/tt0111161/"

In [11]:
data['url'][0]

'https://www.imdb.com/title/tt0114709/'

In [23]:
# movie_poster_fetcher(data['url'][0])

In [ ]:
streamlit run C:\Users\USER\anaconda3\lib\site-packages\ipykernel_launcher.py

In [5]:
def get_movie_info(imdb_link):
    url_data = requests.get(imdb_link).text
    s_data = BeautifulSoup(url_data, 'html.parser')
    imdb_content = s_data.find("meta", property="og:description")
    movie_descr = imdb_content.attrs['content']
    movie_descr = str(movie_descr).split('.')
    movie_director = movie_descr[0]
    movie_cast = str(movie_descr[1]).replace('With', 'Cast: ').strip()
    movie_story = 'Story: ' + str(movie_descr[2]).strip()+'.'
#     rating = s_data.find("div", class_="AggregateRatingButton__TotalRatingAmount-sc-1ll29m0-3 jkCVKJ")
#     rating = str(rating).split('<div class="AggregateRatingButton__TotalRatingAmount-sc-1ll29m0-3 jkCVKJ')
#     rating = str(rating[1]).split("</div>")
#     rating = str(rating[0]).replace(''' "> ''', '').replace('">', '')

    #movie_rating = 'Total Rating count: '+ rating
    return movie_director,movie_cast,movie_story

In [6]:
get_movie_info(link)

('The Shawshank Redemption: Directed by Frank Darabont',
 'Cast:  Tim Robbins, Morgan Freeman, Bob Gunton, William Sadler',
 'Story: Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.')

In [36]:
# page = requests.get("https://www.imdb.com/title/tt0114709/")
# bSoup = BeautifulSoup(page.content, 'html.parser')
# lis_nklist = bSoup.find_all('a')

# for link in links_list:
#     if 'href' in link.attrs and '.mp4' in link.split('@'):
#         print(str(link.attrs['href']) + "\n")